# Summarization of financial data using Hugging Face LLM models
This notebook aims to provide an introduction to documenting an LLM model using the ValidMind Developer Framework. The use case presented is a summarization of financial data (https://huggingface.co/datasets/financial_phrasebank).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate document about the data and model

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace the code below with the code snippet from your project ## 


import validmind as vm

vm.init(
  api_host = "....",
  api_key = "...",
  api_secret = "...",
  project = "..."
)


/Users/anilsorathiya/Library/Caches/pypoetry/virtualenvs/validmind-pPj8dHa5-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2023-09-22 11:29:40,816 - INFO(validmind.api_client): Connected to ValidMind. Project: nlp model sensitivity analysis - Initial Validation (cliop8llc003x32rlklophmdl)


In [3]:
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import textwrap
from tabulate import tabulate
from IPython.display import display, HTML
from rouge import Rouge
import plotly.graph_objects as go
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from pprint import pprint
import torch
import string
import plotly.express as px
import plotly.subplots as sp
from collections import Counter
from itertools import combinations
from dataclasses import dataclass

### CNN data

In [4]:
from datasets import load_dataset
cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')
train_df = cnn_dataset.data['train'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
train_df = train_df[['article','highlights']]
train_df = train_df.head(20)


In [5]:
df = train_df.head(100)
# Load a test dataset with 100 rows only
vm_ds = vm.init_dataset(
    dataset=df,
    text_column="article",
    target_column="highlights",
)

2023-09-22 11:29:48,106 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


In [6]:
text_data_test_plan = vm.run_test_plan("text_data_quality",
                                       dataset=vm_ds,)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anilsorathiya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anilsorathiya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from dataclasses import dataclass
import pandas as pd
from transformers import pipeline

@dataclass
class AbstractSummarization_HuggingFace:
    """
    A VM Model instance wrapper for abstract summarization using HuggingFace Transformers.
    """
    model: any
    tokenizer: any
    predicted_prob_values: list = None

    def __init__(self, model_name=None, model=None, tokenizer=None):
        pipeline_task = "summarization"
        self.model_name = model_name
        self.pipeline_task = pipeline_task
        self.model = pipeline(pipeline_task, model=model, tokenizer=tokenizer)

    def predict(self, texts):
        """
        Generates summaries for the given texts.
        
        Parameters:
        - texts (list): List of texts to be summarized.
        - min_length (int, optional): Minimum length of the produced summary.
        - max_length (int, optional): Maximum length of the produced summary.
        
        Returns:
        - List of summaries.
        """
        
        # min_length = params.get("min_length")
        # max_length = params.get("max_length")
        min_length = None
        max_length = 50
        # If either value is None, don't pass it to the model function
        model_args = {}
        if min_length is not None:
            model_args["min_length"] = min_length
        if max_length is not None:
            model_args["max_length"] = max_length

        summaries = []

        for text in texts:
            data = [str(text)]
            results = self.model(data, **model_args)  # Using ** unpacking to pass arguments conditionally
            results_df = pd.DataFrame(results)
            summary = results_df["summary_text"].values[0] if "summary_text" in results_df.columns else results_df["label"].values[0]
            summaries.append(summary)

        return summaries


    def predict_proba(self):
        """
        Retrieves predicted probabilities after prediction. 
        Note: Not all models provide predicted probabilities.
        """
        if self.predicted_prob_values is None:
            raise ValueError("First run predict method to retrieve predicted probabilities")
        return self.predicted_prob_values

    def description(self):
        """
        Describes the methods available in the class.

        Returns:
        - A string describing the methods.
        """
        desc = (
            "This class provides methods for abstract summarization using HuggingFace Transformers.\n"
            "1. predict: Generates summaries for given texts. Accepts optional min_length and max_length parameters.\n"
            "2. predict_proba: Retrieves predicted probabilities after prediction (if available).\n"
        )
        return desc


In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

summarizer_model = pipeline(
    task="summarization",
    model=model,
    tokenizer = tokenizer,
    min_length=0,
    max_length=60,
    truncation=True,
    model_kwargs={"cache_dir": '/Documents/Huggin_Face/'},
)  # Note: We specify cache_dir to use predownloaded models.

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
df_test = df.head(100)
# Load a test dataset with 100 rows only
vm_test_ds = vm.init_dataset(
    dataset=train_df,
    text_column="article",
    target_column="highlights",
)

2023-09-22 11:29:58,940 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


In [10]:
vm_model = vm.init_model(
    summarizer_model,
    test_ds=vm_test_ds,
)

In [11]:
vm.test_plans.list_plans()

ID,Name,Description
classifier_metrics,ClassifierMetrics,Test plan for sklearn classifier metrics
classifier_validation,ClassifierPerformance,Test plan for sklearn classifier models
classifier_model_diagnosis,ClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests
prompt_validation,PromptValidation,Test plan for prompt validation
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.
time_series_multivariate,TimeSeriesMultivariate,Test plan to perform time series multivariate analysis.
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.


In [12]:
vm.test_plans.describe_plan("summarization_metrics")

ID,Name,Description,Tests
summarization_metrics,SummarizationMetrics,Test plan for Summarization metrics,RougeMetrics (Metric)TokenDisparity (Metric)BleuScore (Metric)BertScore (Metric)ContextualRecall (Metric)


In [13]:
config={
    "rouge_metric": {
        "rouge_metrics": ["rouge-1","rouge-2", "rouge-l"],
    },
}
summarization_metrics = vm.run_test_plan("summarization_metrics", 
                                             model=vm_model,
                                             config=config)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
